In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import pandas as pd

from utils.constants import STAGE_DIR
from utils.dataload import load_data

In [61]:
# Load all dataset
X_all = load_data(
    STAGE_DIR / 'preprocess' / 'X_train.parquet',
    load_func=pd.read_parquet).sort_index()

# Load y dataset
y = load_data(
    STAGE_DIR / 'preprocess' / 'y.parquet',
    load_func=pd.read_parquet).sort_index()

In [62]:
cols = [
    'electricity',
    'toilet',
    'cook_energy',
    'rubbish_disposal',
    'state_wall',
    'state_roof',
    'state_floor',
    'civil_state',
    'education_primary',
    'dwelling_type',
]

In [63]:
real_cols = X_all.columns[X_all.columns.str.contains(r'|'.join(f'^{c}' for c in cols))].sort_values()

In [64]:
X = X_all[real_cols]

In [65]:
df = pd.DataFrame(index=X.index, columns=cols)
var_dir = {}

for col in cols:
    real_col = X.columns[X.columns.str.contains(f'^{col}')].sort_values()
    var_dir[col] = {}
    for i, rc in enumerate(real_col):
        var_dir[col][i + 1] = rc[len(col) + 1:].replace('_', ' ').title()
        X.loc[X[rc].eq(1), rc] = i + 1
    df[col] = X[real_col].sum(1)

C:\Users\gabya\AppData\Local\Temp\ipykernel_10428\2676314179.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[X[rc].eq(1), rc] = i + 1


In [66]:
from pprint import pprint

pprint(var_dir)

{'civil_state': {1: '10-',
                 2: 'Divorced',
                 3: 'Free Or Coupled Union',
                 4: 'Married',
                 5: 'Separated',
                 6: 'Single',
                 7: 'Widow'},
 'cook_energy': {1: 'Electricity', 2: 'Gas', 3: 'No', 4: 'Wood Charcoal'},
 'dwelling_type': {1: 'Other',
                   2: 'Own And Paid',
                   3: 'Own And Paying',
                   4: 'Precarious',
                   5: 'Rented'},
 'education_primary': {1: 'Complete', 2: 'Incomplete'},
 'electricity': {1: 'Cooperative', 2: 'No', 3: 'Private', 4: 'Public'},
 'rubbish_disposal': {1: 'Botan Hollow Or Buried',
                      2: 'Burning',
                      3: 'Other',
                      4: 'Tanker Truck',
                      5: 'Throw Unoccupied Space'},
 'state_floor': {1: 'Bad', 2: 'Good', 3: 'Regular'},
 'state_roof': {1: 'Bad', 2: 'Good', 3: 'Regular'},
 'state_wall': {1: 'Bad', 2: 'Good', 3: 'Regular'},
 'toilet': {1: 'Blac

In [67]:
var_dir.keys()

dict_keys(['electricity', 'toilet', 'cook_energy', 'rubbish_disposal', 'state_wall', 'state_roof', 'state_floor', 'civil_state', 'education_primary', 'dwelling_type'])

In [68]:
list(map(lambda x: '{\tt ' + x + '}', var_dir.keys()))

['{\tt electricity}',
 '{\tt toilet}',
 '{\tt cook_energy}',
 '{\tt rubbish_disposal}',
 '{\tt state_wall}',
 '{\tt state_roof}',
 '{\tt state_floor}',
 '{\tt civil_state}',
 '{\tt education_primary}',
 '{\tt dwelling_type}']

In [69]:
bn_df = pd.concat([df, y], axis=1).astype(float).reset_index(drop=True)
# bn_df = pd.concat([df, y], axis=1).astype(int).reset_index(drop=True).replace(var_dir).replace({'y': {0: 'non-poor', 1: 'poor'}}).replace({0: 'unknown'})

In [70]:
bn_df

,electricity,toilet,cook_energy,rubbish_disposal,state_wall,state_roof,state_floor,civil_state,education_primary,dwelling_type,y
0,4.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,2.0,0.0
1,4.0,4.0,2.0,4.0,3.0,2.0,1.0,3.0,0.0,1.0,1.0
2,4.0,1.0,4.0,1.0,1.0,1.0,1.0,7.0,2.0,1.0,1.0
3,4.0,4.0,2.0,4.0,1.0,1.0,3.0,7.0,0.0,2.0,0.0
4,4.0,4.0,2.0,4.0,2.0,3.0,2.0,5.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2983,4.0,4.0,2.0,4.0,2.0,2.0,2.0,4.0,1.0,2.0,0.0
2984,4.0,4.0,2.0,4.0,2.0,1.0,3.0,3.0,0.0,5.0,0.0
2985,4.0,4.0,2.0,1.0,2.0,2.0,2.0,3.0,1.0,2.0,0.0
2986,4.0,4.0,4.0,1.0,1.0,3.0,1.0,4.0,2.0,2.0,1.0


In [71]:
a = bn_df.replace(var_dir).replace({'y': {0: 'Non-poor', 1: 'Poor'}}).replace({0: 'Unknown'})

In [72]:
a

,electricity,toilet,cook_energy,rubbish_disposal,state_wall,state_roof,state_floor,civil_state,education_primary,dwelling_type,y
0,Public,Septic Tank,Electricity,Botan Hollow Or Buried,Good,Good,Good,10-,Unknown,Own And Paid,Non-poor
1,Public,Septic Tank,Gas,Tanker Truck,Regular,Good,Bad,Free Or Coupled Union,Unknown,Other,Poor
2,Public,Black Hole Or Letrine,Wood Charcoal,Botan Hollow Or Buried,Bad,Bad,Bad,Widow,Incomplete,Other,Poor
3,Public,Septic Tank,Gas,Tanker Truck,Bad,Bad,Regular,Widow,Unknown,Own And Paid,Non-poor
4,Public,Septic Tank,Gas,Tanker Truck,Good,Regular,Good,Separated,Unknown,Other,Poor
...,...,...,...,...,...,...,...,...,...,...,...
2983,Public,Septic Tank,Gas,Tanker Truck,Good,Good,Good,Married,Complete,Own And Paid,Non-poor
2984,Public,Septic Tank,Gas,Tanker Truck,Good,Bad,Regular,Free Or Coupled Union,Unknown,Rented,Non-poor
2985,Public,Septic Tank,Gas,Botan Hollow Or Buried,Good,Good,Good,Free Or Coupled Union,Complete,Own And Paid,Non-poor
2986,Public,Septic Tank,Wood Charcoal,Botan Hollow Or Buried,Bad,Regular,Bad,Married,Incomplete,Own And Paid,Poor


In [73]:
bn_df.to_parquet(STAGE_DIR / 'preprocess' / 'X_BN.parquet')
a.to_parquet(STAGE_DIR / 'preprocess' / 'X_BN_cat.parquet')

In [57]:
a.to_csv(STAGE_DIR / 'preprocess' / 'X_BN_cat.csv')